In [1]:
import pandas as pd
import os, sys

import tensorflow as tf

module_path = os.path.abspath(os.path.join('./..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from settings import *
from notebooks.my_train import *
from src.model import CNN, CNN_nodropout
import librosa

most_shape = (128, 213)
baseline = CNN(most_shape)
baseline.load_model('experiment3_model_5epochs_better')

2023-06-12 10:40:45.560283: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 10:40:45.681752: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-12 10:40:46.358126: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.0: cannot open shared object file: No such file or directory
2023-06-12 10:40:46.358201: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64]

src.model - INFO - Initializing CNN
src.model - INFO - Input shape = (128, 213, 1)
src.model - INFO - CNN Initialized
src.model - INFO - Loading saved model
src.model - INFO - Model loaded from /home/tzag/danigil/dl/guitarCR/models


2023-06-12 10:40:48.070610: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22328 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:3b:00.0, compute capability: 8.6


In [2]:
from src.data.preprocessing import uniform_shape
from src.processing import *

train_datas = []
test_datas = []

instruments = ['Guitar', 'Accordion', 'Violin', 'Piano']
datasets_raw = [pd.read_pickle(os.path.join(METADATA_DIR_PROCESSED_NORMALIZED, f'data_{instrument.lower()}.pkl')) for instrument in instruments]

for i in range(len(datasets_raw)):
    datasets_raw[i] = uniform_shape(datasets_raw[i], most_shape[1])

train_data_guitar, test_data_guitar = train_test_split(datasets_raw[0], augmented=False, split_ratio=0.65)

test_datas.append(test_data_guitar)

src.processing - INFO - Start train test split with split ratio: 0.65
src.processing - INFO - Number of training samples is 1300
src.processing - INFO - Number of testing samples is 700
src.processing - INFO - Train test split completed


In [3]:
train_data_guitar, test_data_guitar = train_test_split(datasets_raw[0], augmented=False, split_ratio=0.65)
test_datas = []
test_datas.append(test_data_guitar)

for i, dataset in enumerate(datasets_raw):
    if i == 0:
        continue

    _, test_data = train_test_split(dataset, augmented=False, split_ratio=0)
    test_datas.append(test_data)

src.processing - INFO - Start train test split with split ratio: 0.65
src.processing - INFO - Number of training samples is 1300
src.processing - INFO - Number of testing samples is 700
src.processing - INFO - Train test split completed
src.processing - INFO - Start train test split with split ratio: 0
src.processing - INFO - Number of training samples is 0
src.processing - INFO - Number of testing samples is 100
src.processing - INFO - Train test split completed
src.processing - INFO - Start train test split with split ratio: 0
src.processing - INFO - Number of training samples is 0
src.processing - INFO - Number of testing samples is 100
src.processing - INFO - Train test split completed
src.processing - INFO - Start train test split with split ratio: 0
src.processing - INFO - Number of training samples is 0
src.processing - INFO - Number of testing samples is 100
src.processing - INFO - Train test split completed


In [5]:
test_instruments = instruments
for test_data, instrument in zip(test_datas, test_instruments):
    X_test = test_data['spectrogram']
    X_test = np.array([x.reshape( (128, most_shape[1], 1) ) for x in X_test])
    y_test = test_data['class_ID']

    y_test_values=y_test
    y_test = np.array(keras.utils.to_categorical(y_test, 10))

    score = baseline.model.evaluate(X_test,y_test)
    print(f'Test score for instrument: {instrument}')
    print('\tTest loss:', score[0])
    print('\tTest accuracy:', score[1])
    print('\tTest precision:', score[2])
    print('\tTest recall:', score[3])
    print('\tTest f1-score:', score[4])

22/22 [==============================] - 0s 8ms/step - loss: 0.2836 - accuracy: 0.9114 - precision: 0.9210 - recall: 0.8973 - fmeasure: 0.9086
Test score for instrument: Guitar
	Test loss: 0.28361475467681885
	Test accuracy: 0.9114285707473755
	Test precision: 0.9209892153739929
	Test recall: 0.8973214626312256
	Test f1-score: 0.908647358417511
4/4 [==============================] - 0s 7ms/step - loss: 0.0057 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - fmeasure: 1.0000
Test score for instrument: Accordion
	Test loss: 0.005708777345716953
	Test accuracy: 1.0
	Test precision: 1.0
	Test recall: 1.0
	Test f1-score: 1.0
4/4 [==============================] - 0s 7ms/step - loss: 0.0032 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - fmeasure: 1.0000
Test score for instrument: Violin
	Test loss: 0.003157589817419648
	Test accuracy: 1.0
	Test precision: 1.0
	Test recall: 1.0
	Test f1-score: 1.0
4/4 [==============================] - 0s 6ms/step - loss: 0.2004 - accuracy

In [8]:
dataset_piano = pd.read_pickle(os.path.join(METADATA_DIR_RAW_OOD, 'data_piano.pkl'))
dataset_guitar = pd.read_pickle(os.path.join(METADATA_DIR_RAW_OOD, 'data_guitar.pkl'))
size=213
dataset_piano['spectrogram'] = dataset_piano['spectrogram'].apply(lambda x: np.pad(x, ((0, 0), (0, size-x.shape[1])), 'constant'))
dataset_guitar['spectrogram'] = dataset_guitar['spectrogram'].apply(lambda x: np.pad(x, ((0, 0), (0, size-x.shape[1])), 'constant'))

test_data = dataset_piano

X_test = test_data['spectrogram']
X_test = np.array([np.pad(x, ((0, 0), (0, size-x.shape[1])), 'constant') for x in dataset_piano['spectrogram']])
y_test = test_data['class_ID']

X_test = np.array([x.reshape( (128, size, 1) ) for x in X_test])

y_test_values=y_test
y_test = np.array(keras.utils.to_categorical(y_test, 10))

score = baseline.model.evaluate(
	x=X_test,
	y=y_test)

print('Piano Test accuracy:', score[1])

test_data = dataset_guitar

X_test = test_data['spectrogram']
X_test = np.array([np.pad(x, ((0, 0), (0, size-x.shape[1])), 'constant') for x in dataset_guitar['spectrogram']])
y_test = test_data['class_ID']

X_test = np.array([x.reshape( (128, size, 1) ) for x in X_test])

y_test_values=y_test
y_test = np.array(keras.utils.to_categorical(y_test, 10))

score = baseline.model.evaluate(
	x=X_test,
	y=y_test)

print('Guitar Test accuracy:', score[1])

4/4 [==============================] - 0s 17ms/step - loss: 62.1479 - accuracy: 0.3600 - precision: 0.4453 - recall: 0.4453 - fmeasure: 0.4453
Piano Test accuracy: 0.36000001430511475
2/2 [==============================] - 0s 11ms/step - loss: 17.3487 - accuracy: 0.4667 - precision: 0.4710 - recall: 0.4710 - fmeasure: 0.4710
Guitar Test accuracy: 0.46666666865348816


In [11]:
y_pred = {}
y_true = {}

from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score
import warnings
warnings.filterwarnings("ignore")

ood2_path = './../data/audio/myood2/splits'
df = pd.DataFrame()

for instrument in os.listdir(ood2_path):
    y_pred[instrument] = []
    # y_pred_3[instrument] = []
    y_true[instrument] = []

    for chord in os.listdir(os.path.join(ood2_path, instrument)):

        #true = file.split('-')[-1][:1]
        true = chord.replace('.wav','')
        assert true in CLASSES
        y_true[instrument].append(true)

        curr_path = os.path.join(ood2_path, instrument, chord)
        y, sr = librosa.load(curr_path, sr=44100, duration=2)

        spectrogram = librosa.feature.melspectrogram(y=y,sr=sr, n_mels=128)
        #spectrogram = librosa.util.normalize(np.log(librosa.feature.melspectrogram(y=y,sr=sr, n_mels=128) + 1e-9))
        spectrogram = np.pad(spectrogram, ((0, 0), (0, most_shape[1]-spectrogram.shape[1])), 'constant')
        spectrogram = spectrogram.reshape((1,)+most_shape+(1,))

        predict_x=baseline.model.predict(spectrogram, batch_size=1, verbose=0)
        predictions = np.argmax(predict_x,axis=1)
        pred = CLASSES[predictions[0]]
        y_pred[instrument].append(pred)
        
        # predict_x_3=experiment3.model.predict(spectrogram, batch_size=1, verbose=0)
        # predictions_3 = np.argmax(predict_x_3,axis=1)
        # pred_3 = CLASSES[predictions_3[0]]
        # y_pred_3[instrument].append(pred_3)

    slen = int((len('#EXP | ACCURACY | RECALL | PRECISION | F1-SCORE') - len(instrument))/2)
    print(f"{'~'*slen}{instrument}{'~'*slen}")
    print(f'#EXP | ACCURACY | RECALL | PRECISION | F1-SCORE')
    print(f"""BASELINE | {'{:.6f}'.format(accuracy_score(y_true[instrument], y_pred[instrument]))} | {'{:.4f}'.format(recall_score(y_true[instrument], y_pred[instrument], average='macro'))} | {'{:.7f}'.format(precision_score(y_true[instrument], y_pred[instrument], average='macro'))} | {'{:.6f}'.format(f1_score(y_true[instrument], y_pred[instrument], average='macro'))}""")
    # print(f"""EXP3 | {'{:.6f}'.format(accuracy_score(y_true[instrument], y_pred_3[instrument]))} | {'{:.4f}'.format(recall_score(y_true[instrument], y_pred_3[instrument], average='macro'))} | {'{:.7f}'.format(precision_score(y_true[instrument], y_pred_3[instrument], average='macro'))} | {'{:.6f}'.format(f1_score(y_true[instrument], y_pred_3[instrument], average='macro'))}""")


~~~~~~~~~~~~~~~~~~~xylophone~~~~~~~~~~~~~~~~~~~
#EXP | ACCURACY | RECALL | PRECISION | F1-SCORE
BASELINE | 0.000000 | 0.0000 | 0.0000000 | 0.000000
~~~~~~~~~~~~~~~~~~~clarinet~~~~~~~~~~~~~~~~~~~
#EXP | ACCURACY | RECALL | PRECISION | F1-SCORE
BASELINE | 0.100000 | 0.1000 | 0.0333333 | 0.050000
~~~~~~~~~~~~~~~~~~~~trumpet~~~~~~~~~~~~~~~~~~~~
#EXP | ACCURACY | RECALL | PRECISION | F1-SCORE
BASELINE | 0.200000 | 0.2000 | 0.0750000 | 0.106667
~~~~~~~~~~~~~~~~~~~~~oboe~~~~~~~~~~~~~~~~~~~~~
#EXP | ACCURACY | RECALL | PRECISION | F1-SCORE
BASELINE | 0.300000 | 0.3000 | 0.1666667 | 0.195238
~~~~~~~~~~~~~~~~~~~~~harp~~~~~~~~~~~~~~~~~~~~~
#EXP | ACCURACY | RECALL | PRECISION | F1-SCORE
BASELINE | 0.100000 | 0.1000 | 0.0200000 | 0.033333
~~~~~~~~~~~~~~~~~tubular_bells~~~~~~~~~~~~~~~~~
#EXP | ACCURACY | RECALL | PRECISION | F1-SCORE
BASELINE | 0.000000 | 0.0000 | 0.0000000 | 0.000000
~~~~~~~~~~~~~~~~~~~~~horn~~~~~~~~~~~~~~~~~~~~~
#EXP | ACCURACY | RECALL | PRECISION | F1-SCORE
BASELINE | 0.200000 